# MMEngine 快速上手

参考：[MMEngine 快速上手](https://mmengine.readthedocs.io/zh-cn/latest/get_started/15_minutes.html)

## 构建模型

首先，需要构建 MMEngine 模型，约定这个模型应当继承 {class}`~mmengine.model.BaseModel`，并且其 `forward` 方法除了接受来自数据集的若干参数外，还需要接受额外的参数 `mode`：对于训练，需要 `mode` 接受字符串 `"loss"`，并返回包含 `"loss"` 字段的字典；对于验证，需要 `mode` 接受字符串 `"predict"`，并返回同时包含预测信息和真实信息的结果。

In [1]:
import set_env

In [2]:
import torch.nn.functional as F
import torch
from torch import nn
import torchvision
from mmengine.model import BaseModel

class MMResNet50(BaseModel):
    def __init__(self, data_preprocessor: dict|nn.Module|None = None,
                 init_cfg: dict|None = None):
        super().__init__(data_preprocessor=data_preprocessor, init_cfg=init_cfg)
        self.resnet = torchvision.models.resnet50()

    def forward(self, inputs: torch.Tensor,
                data_samples: list|list = None,
                mode: str = 'tensor') -> dict[str, torch.Tensor] | list:
        x = self.resnet(inputs)
        if mode == 'loss':
            return {'loss': F.cross_entropy(x, data_samples)}
        elif mode == 'predict':
            return x, data_samples
        else:
            return x

/media/pc/data/lxw/envs/anaconda3x/envs/xxx/lib/python3.12/site-packages/mmengine/optim/optimizer/zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \


## 构建数据集和数据加载器

其次，需要构建训练和验证所需要的数据集 ({class}`~torch.utils.data.Dataset`)和数据加载器 ({class}`~torch.utils.data.DataLoader`)。 对于基础的训练和验证功能，可以直接使用符合 PyTorch 标准的数据加载器和数据集。

In [3]:
from pathlib import Path

temp_dir = Path(".temp")
temp_dir.mkdir(exist_ok=True) # 创建缓存目录

In [4]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

norm_cfg = dict(mean=[0.491, 0.482, 0.447], std=[0.202, 0.199, 0.201])
train_dataloader = DataLoader(
    batch_size=32,
    shuffle=True,
    dataset=torchvision.datasets.CIFAR10(
        temp_dir/'data/cifar10',
        train=True,
        download=True,
        transform=transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(**norm_cfg)
        ]))
)
val_dataloader = DataLoader(
    batch_size=32,
    shuffle=False,
    dataset=torchvision.datasets.CIFAR10(
        temp_dir/'data/cifar10',
        train=False,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(**norm_cfg)
        ]))
)

Files already downloaded and verified
Files already downloaded and verified


## 构建评测指标

为了进行验证和测试，需要定义模型推理结果的评测指标。

约定评测指标需要继承 {class}`~mmengine.evaluator.BaseMetric`，并实现 `process` 和 `compute_metrics` 方法。其中 `process` 方法接受数据集的输出和模型 `mode="predict"` 时的输出，此时的数据为单个批次的数据，对这一批次的数据进行处理后，保存信息至 `self.results` 属性。而 `compute_metrics` 接受 `results` 参数，这一参数的输入为 `process` 中保存的所有信息（如果是分布式环境，`results` 中为已收集的，包括各个进程 `process` 保存信息的结果），利用这些信息计算并返回保存有评测指标结果的字典

In [5]:
from typing import Any, Sequence
from mmengine.evaluator import BaseMetric


class Accuracy(BaseMetric):
    def process(self, data_batch: Any, data_samples: Sequence[dict])->None:
        """
        处理一批数据样本及其预测结果。处理后的结果应存储在`self.results`中，这将在所有批次处理完毕后用于计算指标。

        Args:
            data_batch: 从数据加载器获取的一批数据。
            data_samples: 模型输出的一批结果。
        """
        score, gt = data_samples
        # 将一个批次的中间结果保存至 `self.results`
        self.results.append({
            'batch_size': len(gt),
            'correct': (score.argmax(dim=1) == gt).sum().cpu(),
        })

    def compute_metrics(self, results):
        total_correct = sum(item['correct'] for item in results)
        total_size = sum(item['batch_size'] for item in results)
        # 返回保存有评测指标结果的字典，其中键为指标名称
        return dict(accuracy=100 * total_correct / total_size)

## 构建执行器并执行任务

最后，利用构建好的模型，数据加载器，评测指标构建执行器 ({class}`~mmengine.runner.Runner`)，同时在其中配置 优化器、工作路径、训练与验证配置等选项，即可通过调用执行器的 {meth}`~mmengine.runner.Runner.train` 方法启动训练：

In [6]:
from torch.optim import SGD
from mmengine.runner import Runner

runner = Runner(
    # 用以训练和验证的模型，需要满足特定的接口需求
    model=MMResNet50(),
    # 工作路径，用以保存训练日志、权重文件信息
    work_dir=temp_dir/'./work_dir',
    # 训练数据加载器，需要满足 PyTorch 数据加载器协议
    train_dataloader=train_dataloader,
    # 优化器包装，用于模型优化，并提供 AMP、梯度累积等附加功能
    optim_wrapper=dict(optimizer=dict(type=SGD, lr=0.001, momentum=0.9)),
    # 训练配置，用于指定训练周期、验证间隔等信息
    train_cfg=dict(by_epoch=True, max_epochs=5, val_interval=1),
    # 验证数据加载器，需要满足 PyTorch 数据加载器协议
    val_dataloader=val_dataloader,
    # 验证配置，用于指定验证所需要的额外参数
    val_cfg=dict(),
    # 用于验证的评测器，这里使用默认评测器，并评测指标
    val_evaluator=dict(type=Accuracy),
)

runner.train()

11/22 16:43:51 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 20:50:58) [GCC 12.3.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1560666916
    GPU 0: NVIDIA GeForce RTX 3090
    GPU 1: NVIDIA GeForce RTX 2080 Ti
    CUDA_HOME: /media/pc/data/lxw/envs/anaconda3x/envs/xxx
    NVCC: Cuda compilation tools, release 12.6, V12.6.20
    GCC: gcc (conda-forge gcc 12.4.0-0) 12.4.0
    PyTorch: 2.5.0
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.5.3 (Git Hash 66f0cb9eb66affd2da3bf5f8d897376f04aae6af)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  -

MMResNet50(
  (data_preprocessor): BaseDataPreprocessor()
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=Tr